In [1]:
from models import MotionTransformer

In [7]:
encoder = MotionTransformer(
        input_feats=264,
        num_frames=360,
        num_layers=8,
        latent_dim=512,
        no_clip=False,
        no_eff=False)

/home/ltdoanh/.local/lib/python3.10/site-packages/torch/nn/modules/transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


In [8]:
from utils.get_opt import get_opt

In [9]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
opt_path = "./checkpoints/beat/beat_diffusion_v1/opt.txt"
opt = get_opt(opt_path, device)
opt.do_denoise = True

Reading ./checkpoints/beat/beat_diffusion_v1/opt.txt


In [20]:
import numpy as np
from os.path import join as pjoin
from trainers import DDPMTrainer

mean = np.load(pjoin(opt.meta_dir, 'mean.npy'))
std = np.load(pjoin(opt.meta_dir, 'std.npy'))
trainer = DDPMTrainer(opt, encoder)
trainer.load(pjoin(opt.model_dir, opt.which_epoch + '.tar'))

trainer.eval_mode()
trainer.to(opt.device)

result_dict = {}
with torch.no_grad():
        caption = ["hi how are u"]
        m_lens = torch.LongTensor([360]).to(device)
        pred_motions = trainer.generate(caption, m_lens, 264)
        motion = pred_motions[0].cpu().numpy()
        print(motion.shape)
opt.result_dir = "/home/ltdoanh/ldtan/Motion_Diffusion/checkpoints"
np.save(pjoin(opt.result_dir, 'motion.npy'), motion)

100%|██████████| 1000/1000 [00:15<00:00, 62.94it/s]

(360, 264)
